# **Constructing qubit Hamiltonians from hickle dump files**

This folder contains various Hamiltonian sizes derived from the different active space sizes to model the dissocitation of water on a magnesium slab. 

$$ Mg + H_2O \rightarrow Mg(OH_{ads})(H_{ads}) $$

<p align="center">
  <img src="Mg_H2O.jpg" alt="Alternative text" />
</p>

Note that this folder contains Hamiltonian files for both the reactant (Step 1) and product (step 23) geometries. The Hamiltonians vary in sizes depending on the active space sizes, which were chosen based on classical preprocessing calculations. Additionally, within each active space, there are 16 Hamiltonians corresponding to the different K-points using the twist average method. Hence, to estimate the ground state energy for selected active space for a particular geometry (reactatnt or product), we need to perform 16 energy estimations. 

The Hamiltonians are saved in the hickle dump file format (similar to fci dump). We can use qiskit0nature to reconstruct the second quantization fermionic representation of these Hamiltonians, and then map them into qubit representation.


**Qiskit minimum requirements:**

```
{'qiskit-terra': '0.18.2', 'qiskit-aer': '0.8.2', 'qiskit-ignis': '0.6.0', 'qiskit-ibmq-provider': '0.16.0', 
 'qiskit-aqua': '0.9.5', 'qiskit': '0.29.1', 'qiskit-nature': '0.2.1', 'qiskit-finance': None, 
'qiskit-optimization': None, 'qiskit-machine-learning': None}```

## **Example: Hamiltonian for the reactant (Step 1) geometry with AS size of 4 orbitals at gamma point (K=0)**


In [10]:
import hickle
import numpy as np 
from Hamiltonian_Reconstruction import Hamiltonian_Reconstruction

In [2]:
hickle_file_name = 'Step1_dzv_NO_HFtrans_k0_4MO.h5'
gamma_point = True ## Gamma point files are those with k0 in the file name!!! ***IMPORTANT***
mo_ints = hickle.load(hickle_file_name)
H_info = Hamiltonian_Reconstruction(mo_ints, gamma_point)


particle number: ParticleNumber:
	8 SOs
	2 alpha electrons
		orbital occupation: [1. 1. 0. 0.]
	2 beta electrons
		orbital occupation: [1. 1. 0. 0.]


### **2nd quantized fermionic Hamiltonian**

See Qiskit API here to understand the format of this: https://qiskit.org/documentation/nature/stubs/qiskit_nature.operators.second_quantization.FermionicOp.html#qiskit_nature.operators.second_quantization.FermionicOp  

In [3]:
second_quant_fermionic_hamiltonian = H_info.fermionic_hamiltonian_getter()
second_quant_fermionic_hamiltonian.to_list()

[('+_0 +_1 -_2 -_3', (-0.08680297934883208+0j)),
 ('+_0 -_1 +_2 -_3', (0.06628271546226322+0j)),
 ('+_0 -_1 -_2 +_3', (0.02052026388656879+0j)),
 ('+_0 -_1 +_4 -_5', (0.05832150433600469+0j)),
 ('+_0 -_1 +_4 -_6', (-0.02255764210403403+0j)),
 ('+_0 -_1 +_4 -_7', (-0.0009128334438275086+0j)),
 ('+_0 -_1 -_4 +_5', (-0.0583215043360047+0j)),
 ('+_0 -_1 -_4 +_6', (0.02255764210403403+0j)),
 ('+_0 -_1 -_4 +_7', (0.0009128334438275425+0j)),
 ('+_0 -_1 +_5 -_6', (-0.0017862575421132387+0j)),
 ('+_0 -_1 +_5 -_7', (0.017568555281110072+0j)),
 ('+_0 -_1 -_5 +_6', (0.0017862575421132554+0j)),
 ('+_0 -_1 -_5 +_7', (-0.01756855528111007+0j)),
 ('+_0 -_1 +_6 -_7', (-0.06503987663960867+0j)),
 ('+_0 -_1 -_6 +_7', (0.06503987663960868+0j)),
 ('+_0 -_1', (0.001570634040953301+0j)),
 ('+_0 -_1 +_7 -_7', (-0.005715068984328617+0j)),
 ('+_0 -_1 +_6 -_6', (0.004530550756403386+0j)),
 ('+_0 -_1 +_5 -_5', (-3.260730955818241e-05+0j)),
 ('+_0 -_1 +_4 -_4', (-0.001538026615363334+0j)),
 ('+_0 -_1 +_3 -_3', (0.

## **2nd quantized qubit Hamiltonian**

This will be saved in PauliSumOp format (Qiskit's formatting)

In [4]:
qubit_hamiltonian = H_info.qubit_hamiltonian_getter()


We can convert this into a list of Pauli strings and their corresponding weights as follow

In [7]:
### Pauli strings that make up the Hamiltonian 
H = qubit_hamiltonian.to_pauli_op()
Paulis, Weights = [], []   
for term in H:
    Paulis.append(str( term.primitive) )
    Weights.append(term.coeff )

### **Hamiltonian in OpenFermion format**

In [14]:
from openfermion_ham_getter import list_weights_and_paulis_to_openfermion_Hamiltonian

openfermion_ham = list_weights_and_paulis_to_openfermion_Hamiltonian(Weights, Paulis)
np.save('OpenFermion_H' + '_for_' + hickle_file_name , openfermion_ham)
openfermion_ham

['-2.983913816185503 []',
 '-0.2998227832491841 [Z0]',
 '0.22682562708454485 [Z1]',
 '-0.30234001456188714 [Z0 Z1]',
 '0.18692398308271604 [Z2]',
 '0.1954669968005 [Z0 Z2]',
 '0.1583053703037252 [Z1 Z2]',
 '0.2362642157173093 [Z0 Z1 Z2]',
 '-0.2998227832491842 [Z3]',
 '0.14893894023513002 [Z0 Z3]',
 '0.13239251724149229 [Z0 Z1 Z3]',
 '0.14285261072475627 [Z2 Z3]',
 '0.13132565081525863 [Z1 Z2 Z3]',
 '0.22682562708454485 [Z4]',
 '-0.3023400145618871 [Z3 Z4]',
 '0.13239251724149229 [Z0 Z3 Z4]',
 '0.13826673861288055 [Z0 Z1 Z3 Z4]',
 '0.1287788320392311 [Z2 Z3 Z4]',
 '0.13472269521370742 [Z1 Z2 Z3 Z4]',
 '0.18692398308271638 [Z5]',
 '0.14285261072475627 [Z0 Z5]',
 '0.1287788320392311 [Z0 Z1 Z5]',
 '0.1457366574398498 [Z2 Z5]',
 '0.13190795532035093 [Z1 Z2 Z5]',
 '0.19546699680050003 [Z3 Z5]',
 '0.1583053703037257 [Z4 Z5]',
 '0.13132565081525863 [Z0 Z4 Z5]',
 '0.13472269521370742 [Z0 Z1 Z4 Z5]',
 '0.13190795532035093 [Z2 Z4 Z5]',
 '0.14376416683259277 [Z1 Z2 Z4 Z5]',
 '0.23626421571730932 